# HaploHyped VarAwareML - Interactive Tutorial

This notebook demonstrates the complete pipeline from VCF processing to ML-ready haplotype data.

## Overview

**HaploHyped VarAwareML** is a high-performance genomic data pipeline that:
- Converts VCF files to compressed HDF5 format
- Encodes reference genomes efficiently
- Provides PyTorch datasets for deep learning

### Performance Highlights
- **6.5x** compression ratio with Blosc2
- **559K** variants/sec parsing speed
- **342K** records/sec read performance

## Setup

First, let's import the necessary libraries and set up our environment.

In [ ]:
import os
import sys
from pathlib import Path
import numpy as np
import h5py
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Configure paths
DATA_DIR = Path("../tests/data")
OUTPUT_DIR = Path("../output/tutorial")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"\nTest data files:")
for f in sorted(DATA_DIR.glob("*")):
    print(f"  - {f.name}")

## Step 1: VCF to HDF5 Conversion

Convert phased VCF files to compressed HDF5 format for efficient storage and access.

### Why HDF5?
- **Compression**: 6.5x reduction in file size
- **Random Access**: Fast slicing and indexing
- **Structured Data**: Preserve variant annotations and phasing

In [ ]:
from haplohyped.vcf_to_h5 import VCFtoHDF5Converter

# Initialize converter
converter = VCFtoHDF5Converter(
    cohort_name="tutorial_cohort",
    vcf_dir=str(DATA_DIR),
    out_dir=str(OUTPUT_DIR),
    sample_list_path=str(DATA_DIR / "ipscs_samples_test.txt"),
    cores=2,
    cxx_threads=1
)

print(f"Processing {len(converter.donor_ids)} samples")
print(f"Samples: {converter.donor_ids}")
print(f"\nOutput file: {OUTPUT_DIR / 'tutorial_cohort.h5'}")

# Run conversion (requires C++ module built with ./build.sh)
# Uncomment to execute:
# converter.run()

## Step 2: Inspect VCF Data

Let's examine the synthetic VCF file to understand the data structure.

In [ ]:
import gzip

vcf_path = DATA_DIR / "chr22.filtered.vcf.gz"

# Read first few variants
variants = []
with gzip.open(vcf_path, 'rt') as f:
    for line in f:
        if line.startswith('#'):
            if line.startswith('#CHROM'):
                headers = line.strip().split('\t')
            continue
        variants.append(line.strip().split('\t'))
        if len(variants) >= 10:
            break

print("VCF Headers:")
print("  " + ", ".join(headers[:9]))
print(f"  Samples: {', '.join(headers[9:])}")
print(f"\nFirst 5 variants:")
for i, var in enumerate(variants[:5], 1):
    print(f"  {i}. {var[0]}:{var[1]} {var[3]}->{var[4]} (INFO: {var[7][:50]}...)")
    print(f"     Genotypes: {', '.join(var[9:])}")

## Step 3: Reference Genome Encoding

Encode the reference genome into one-hot format for variant-aware sequence generation.

In [ ]:
from haplohyped.fasta_encoder import ReferenceGenome

ref_output = OUTPUT_DIR / "reference_genome.h5"

print(f"Input FASTA: {DATA_DIR / 'chr22.fasta'}")
print(f"Output HDF5: {ref_output}")
print("\nEncoding format: One-hot (A, C, G, T channels)")

# Uncomment to run:
# ref_genome = ReferenceGenome(
#     fasta_file=str(DATA_DIR / "chr22.fasta"),
#     output_dir=str(OUTPUT_DIR / "tmp_ref")
# )
# ref_genome.load_genome_parallel()

## Step 4: Visualize Data Distribution

Analyze the allele frequency distribution in our synthetic VCF.

In [ ]:
# Parse VCF to extract allele frequencies
import re

allele_freqs = []
allele_counts = []

with gzip.open(vcf_path, 'rt') as f:
    for line in f:
        if line.startswith('#'):
            continue
        fields = line.strip().split('\t')
        info = fields[7]
        
        # Extract AF and AC
        af_match = re.search(r'AF=([0-9.]+)', info)
        ac_match = re.search(r'AC=(\d+)', info)
        
        if af_match:
            allele_freqs.append(float(af_match.group(1)))
        if ac_match:
            allele_counts.append(int(ac_match.group(1)))

# Plot distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Allele Frequency Distribution
axes[0].hist(allele_freqs, bins=30, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Allele Frequency (AF)', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Allele Frequency Distribution', fontsize=14, fontweight='bold')
axes[0].axvline(np.mean(allele_freqs), color='red', linestyle='--', 
                label=f'Mean: {np.mean(allele_freqs):.3f}')
axes[0].legend()

# Allele Count Distribution
axes[1].hist(allele_counts, bins=range(0, max(allele_counts)+2), 
             edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Allele Count (AC)', fontsize=12)
axes[1].set_ylabel('Count', fontsize=12)
axes[1].set_title('Allele Count Distribution', fontsize=14, fontweight='bold')
axes[1].axvline(np.mean(allele_counts), color='red', linestyle='--',
                label=f'Mean: {np.mean(allele_counts):.2f}')
axes[1].legend()

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'allele_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Total variants analyzed: {len(allele_freqs)}")
print(f"AF range: {min(allele_freqs):.3f} - {max(allele_freqs):.3f}")
print(f"AC range: {min(allele_counts)} - {max(allele_counts)}")

## Step 5: PyTorch Dataset Creation

Create a PyTorch-compatible dataset for deep learning applications.

In [ ]:
from datasets.haplotype_dataset import RandomHaplotypeDataset
from torch.utils.data import DataLoader

print("Dataset Configuration:")
print("=" * 60)
print(f"  BED file:       {DATA_DIR / 'test_regions.bed'}")
print(f"  Genotype file:  {OUTPUT_DIR / 'tutorial_cohort.h5'}")
print(f"  Reference file: {ref_output}")
print(f"  Sequence length: 1000 bp")
print(f"  Batch size:      32")
print("\nNote: Requires HDF5 files from Steps 1-2")

# Uncomment to create dataset (after running conversion steps):
# dataset = RandomHaplotypeDataset(
#     bed_file=str(DATA_DIR / "test_regions.bed"),
#     hdf5_genotype_file=str(OUTPUT_DIR / "tutorial_cohort.h5"),
#     hdf5_reference_file=str(ref_output),
#     samples_file=str(DATA_DIR / "ipscs_samples_test.txt"),
#     seq_length=1000,
#     batch_size=32
# )
#
# dataloader = DataLoader(dataset, batch_size=8, num_workers=2)
#
# # Show sample batch
# for hap1, hap2 in dataloader:
#     print(f"Batch shape: hap1={hap1.shape}, hap2={hap2.shape}")
#     print(f"Data type: {hap1.dtype}")
#     break

## Step 6: Performance Benchmarking

Benchmark the HDF5 compression and read performance.

In [ ]:
import time
import hdf5plugin

# Create test data
n_variants = 10000
n_samples = 100
test_data = np.random.randint(0, 3, size=(n_variants, n_samples), dtype=np.int8)

# Benchmark compression
print("Compression Benchmark")
print("=" * 60)

# Uncompressed
start = time.time()
with h5py.File(OUTPUT_DIR / "bench_uncompressed.h5", "w") as f:
    f.create_dataset("data", data=test_data)
uncompressed_time = time.time() - start
uncompressed_size = os.path.getsize(OUTPUT_DIR / "bench_uncompressed.h5")

# Blosc2 compressed
start = time.time()
with h5py.File(OUTPUT_DIR / "bench_compressed.h5", "w") as f:
    f.create_dataset("data", data=test_data, **hdf5plugin.Blosc2())
compressed_time = time.time() - start
compressed_size = os.path.getsize(OUTPUT_DIR / "bench_compressed.h5")

print(f"Uncompressed: {uncompressed_size/1024:.1f} KB ({uncompressed_time:.3f}s)")
print(f"Compressed:   {compressed_size/1024:.1f} KB ({compressed_time:.3f}s)")
print(f"Ratio:        {uncompressed_size/compressed_size:.2f}x")

# Benchmark read performance
print(f"\nRead Performance Benchmark")
print("=" * 60)

# Read uncompressed
start = time.time()
with h5py.File(OUTPUT_DIR / "bench_uncompressed.h5", "r") as f:
    data = f["data"][:]
read_uncompressed = time.time() - start

# Read compressed
start = time.time()
with h5py.File(OUTPUT_DIR / "bench_compressed.h5", "r") as f:
    data = f["data"][:]
read_compressed = time.time() - start

print(f"Uncompressed read: {read_uncompressed:.3f}s ({n_variants/read_uncompressed:.0f} records/sec)")
print(f"Compressed read:   {read_compressed:.3f}s ({n_variants/read_compressed:.0f} records/sec)")
print(f"Speedup:           {read_uncompressed/read_compressed:.2f}x")

# Cleanup
os.remove(OUTPUT_DIR / "bench_uncompressed.h5")
os.remove(OUTPUT_DIR / "bench_compressed.h5")

## Summary

This tutorial demonstrated:

1. **VCF to HDF5 Conversion**: Efficient compression and storage
2. **Reference Encoding**: One-hot encoding for variant-aware sequences
3. **Data Visualization**: Allele frequency and count distributions
4. **PyTorch Integration**: ML-ready dataset creation
5. **Performance Benchmarking**: 6.5x compression, fast random access

### Next Steps

- Build the C++ module: `./build.sh`
- Run the full pipeline with your own data
- Integrate with deep learning models
- Explore advanced features in the documentation

### Resources

- [GitHub Repository](https://github.com/Jaureguy760/HaploHyped-VarAwareML)
- [Documentation](docs/)
- [Example Scripts](examples/)